In [213]:
import sys
!{sys.executable} -m pip install ilupp

In [505]:
import numpy as np
import numpy.matlib
import math
from typing import Tuple
from collections import namedtuple
from scipy.sparse import csr_matrix, csc_matrix
from scipy.sparse.linalg import cg
import copy
import ilupp


Hexahedron = namedtuple('Hexahedron', ['ke_lambda', 'ke_mu', 'fe_lambda', 'fe_mu'])

class Homogenization:
    def __init__(self, 
        cell_len_x: int, 
        cell_len_y: int, 
        cell_len_z: int, 
        lambda_: Tuple[float, float], 
        mu: Tuple[float, float], 
        voxel: np.array
    ):
        self.cell_len_x = cell_len_x
        self.cell_len_y = cell_len_y
        self.cell_len_z = cell_len_z

        self.lambda_ = lambda_
        self.mu = mu
        self.voxel = voxel

        # 6 x 6 Constitutive Tensor
        self.constitutuve_tensor = np.zeros((6, 6))

    def homogenize(self):
        ## Initialize
        # Begin discretization
        n_el_x, n_el_y, n_el_z = self.voxel.shape
        n_el = n_el_x * n_el_y * n_el_z
        dx = self.cell_len_x / n_el_x
        dy = self.cell_len_y / n_el_y
        dz = self.cell_len_z / n_el_z

        # The stiffness matrix is broken down into two parts, one for lambda
        # and the other for mu, we'll separately compute them and then
        # combine.
        ke_lambda, ke_mu, fe_lambda, fe_mu = self._compute_hexahedron(
            dx / 2, dy / 2, dz / 2)

        element_degrees_of_freedom = self._compute_degrees_of_freedom(n_el)

        # Impose Periodic Boundary Conditions
        unique_nodes_tensor = self._compute_unique_nodes(n_el)

        # Make degrees of freedom that we can index into with our unique nodes tensor
        element_degrees_of_freedom = self._compute_unique_degrees_of_freedom(element_degrees_of_freedom, unique_nodes_tensor)
        n_degrees_of_freedom = 3 * n_el

        # Material properties for the composite vectors. We just sum them
        self.lambda_ = self.lambda_[0] * ((self.voxel == 1).astype(int)) + \
            self.lambda_[1] * ((self.voxel == 2).astype(int))
        self.mu = self.mu[0] * ((self.voxel == 1).astype(int)) + \
            self.mu[1] * ((self.voxel == 2).astype(int))

        # Now, begin assembling the global stiffness matrix and load vectors
        K = self._assemble_K(
            element_degrees_of_freedom, ke_lambda, ke_mu, n_degrees_of_freedom)

        F = self._assemble_load(
            element_degrees_of_freedom, fe_lambda, fe_mu, n_el, n_degrees_of_freedom)

        # Solve for displacement by PCG method
        X = self._compute_displacement(K, F, element_degrees_of_freedom, n_degrees_of_freedom)

        #### HOMOGENIZATION STARTS HERE
        # The displacement vectors corresponding to unit strain cases
        X0 = np.zeros((n_el, 24, 6))

        # Element displacements for the six unit strains
        X0_e = np.zeros((24, 6))

        # Fix DOF nodes [1 2 3 5 6 12]
        # The exact ratio doesn't matter, it's reflected in the load vector
        ke = ke_mu + ke_lambda
        fe = fe_mu + fe_lambda

        indices = np.concatenate([np.array([3]), np.arange(6, 11), np.arange(12, 24)])
        X0_e[indices, :] = np.linalg.lstsq(ke[indices, :][:, indices], fe[indices, :])[0]
        X0[:,:,0] = np.kron(X0_e[:,0].transpose(), np.ones((n_el,1))) # epsilon0_11 = (1,0,0,0,0,0)
        X0[:,:,1] = np.kron(X0_e[:,1].transpose(), np.ones((n_el,1))) # epsilon0_22 = (0,1,0,0,0,0)
        X0[:,:,2] = np.kron(X0_e[:,2].transpose(), np.ones((n_el,1))) # epsilon0_33 = (0,0,1,0,0,0)
        X0[:,:,3] = np.kron(X0_e[:,3].transpose(), np.ones((n_el,1))) # epsilon0_12 = (0,0,0,1,0,0)
        X0[:,:,4] = np.kron(X0_e[:,4].transpose(), np.ones((n_el,1))) # epsilon0_23 = (0,0,0,0,1,0)
        X0[:,:,5] = np.kron(X0_e[:,5].transpose(), np.ones((n_el,1))) # epsilon0_13 = (0,0,0,0,0,1)

        # Fill Constitutive Tensor
        volume = self.cell_len_x * self.cell_len_y * self.cell_len_z

        for i in range(6):
            for j in range(6):
                indices_i = element_degrees_of_freedom - 1
                indices_j = element_degrees_of_freedom - 1

                sum_L_lhs = np.matmul(X0[:, :, i] - X[indices_i, i], ke_lambda)
                sum_L_rhs = X0[:, :, j] - X[indices_j, j]
                sum_L = sum_L_lhs * sum_L_rhs

                sum_M_lhs = np.matmul(X0[:, :, i] - X[indices_i, i], ke_mu)
                sum_M_rhs = X0[:, :, j] - X[indices_j, j]
                sum_M = sum_M_lhs * sum_M_rhs


                sum_L = np.sum(sum_L, 1)
                sum_L = sum_L.reshape(n_el_x, n_el_y, n_el_z)
                sum_M = np.sum(sum_M, 1)
                sum_M = sum_M.reshape(n_el_x, n_el_y, n_el_z)
                
                self.constitutuve_tensor[i][j] = 1 / volume * np.sum(np.sum(np.sum(self.lambda_ * sum_L + self.mu * sum_M)))

        print(self.constitutuve_tensor)
        print('Homogenization complete!')
        
    def sparse(self, i, j, v, m, n):
        """
        Create and compressing a matrix that have many zeros
        Parameters:
            i: 1-D array representing the index 1 values 
                Size n1
            j: 1-D array representing the index 2 values 
                Size n1
            v: 1-D array representing the values 
                Size n1
            m: integer representing x size of the matrix >= n1
            n: integer representing y size of the matrix >= n1
        Returns:
            s: 2-D array
                Matrix full of zeros excepting values v at indexes i, j
        """
        return csr_matrix((v, (i, j)), shape=(m, n))

    def _compute_degrees_of_freedom(self, n_el):
        n_el_x, n_el_y, n_el_z = self.voxel.shape

        self.number_of_nodes = (1 + n_el_x) * (1 + n_el_y) * (1 + n_el_z)

        # Applying the periodic boundary condition for periodic 
        # volmes. Here we apply the node numbers and degrees of freedom for 
        # this approximation.
        node_numbers = np.c_[np.arange(1, (self.number_of_nodes) + 1)]
        node_numbers = node_numbers.reshape(
            1 + n_el_x, 1 + n_el_y, 1 + n_el_z)
        
        degrees_of_freedom = 3 * node_numbers[0:node_numbers.shape[0]-1, 0:node_numbers.shape[1]-1, 0:node_numbers.shape[2]-1] + 1
        degrees_of_freedom = degrees_of_freedom.reshape(n_el, 1)


        mid = 3 * n_el_x + np.array([3, 4, 5, 0, 1, 2])
        add_x = np.concatenate((np.array([0, 1, 2]), mid, np.array([-3, -2, -1])))
        add_xy = 3 * (n_el_y + 1) * (n_el_x + 1) + add_x

        element_degrees_of_freedom = np.matlib.repmat(
            degrees_of_freedom, 1, 24) + np.matlib.repmat(
                np.concatenate([add_x, add_xy]), n_el, 1)

        return element_degrees_of_freedom

    def _compute_unique_nodes(self, n_el):
        n_el_x, n_el_y, n_el_z = self.voxel.shape
        number_unique_nodes = n_el

        unique_nodes_tensor = np.arange(1, number_unique_nodes + 1)
        unique_nodes_tensor = unique_nodes_tensor.reshape(n_el_x, n_el_y, n_el_z)
        back_border_cols = unique_nodes_tensor[:, :, 0]

        unt_indices = []
        for i, matrix in enumerate(unique_nodes_tensor):
            unt_indices.append(np.append(matrix, back_border_cols[i][...,None], 1))
        
        unique_nodes_tensor = np.array(unt_indices)

        left_border_cols = unique_nodes_tensor[:, 0, :]

        unt_indices = []
        for i, matrix in enumerate(unique_nodes_tensor):
            unt_indices.append(np.vstack([matrix, left_border_cols[i]]))
        
        unique_nodes_tensor = np.array(unt_indices)
        unique_nodes_tensor = np.concatenate((unique_nodes_tensor, unique_nodes_tensor[0][None]), axis=0)
        return unique_nodes_tensor

    def _assemble_K(self, edof, ke_lambda, ke_mu, ndof):
        # Index vectors
        stiffness_index_i = self._flat_1d(np.kron(edof, np.ones((24, 1))).transpose())
        stiffness_index_i -= 1
        stiffness_index_j = self._flat_1d(np.kron(edof, np.ones((1, 24))).transpose())
        stiffness_index_j -= 1
        stiffness_entries = np.matmul(self._flat_2d(ke_lambda), self._flat_2d(self.lambda_).conj().transpose())  + \
            np.matmul(self._flat_2d(ke_mu), self._flat_2d(self.mu).conj().transpose())

        stiffness_entries = self._flat_1d(stiffness_entries)

        K = self.sparse(
            stiffness_index_i, 
            stiffness_index_j,
            stiffness_entries, 
            ndof, 
            ndof
        )
        K = 1/2 * (K + K.transpose())
        return K

    def _assemble_load(self, edof, fe_lambda, fe_mu, n_el, ndof):
        load_index_i = self._flat_1d(np.matlib.repmat(edof.transpose(), 6, 1))
        load_index_i -= 1
        load_index_j = self._flat_1d(np.concatenate((
            np.ones((24, n_el)),
            2 * np.ones((24, n_el)),
            3 * np.ones((24, n_el)),
            4 * np.ones((24, n_el)),
            5 * np.ones((24, n_el)),
            6 * np.ones((24, n_el))
        )))
        load_index_j -= 1
        load_entries = np.matmul(
            self._flat_2d(fe_lambda), 
            self._flat_2d(self.lambda_).conj().transpose(), 
        ) + np.matmul(
            self._flat_2d(fe_mu), 
            self._flat_2d(self.mu).conj().transpose()
        )
        load_entries = self._flat_1d(load_entries)
        F = self.sparse(
            load_index_i, load_index_j, load_entries, ndof, 6)

        return F

    def _compute_unique_degrees_of_freedom(self, edof, unt):
        _dof = np.zeros((3 * self.number_of_nodes, 1))
        _uniq = unt.reshape(np.prod(unt.shape))
        for i in range(0, _dof.shape[0], 3):
            idx = i // 3
            _dof[i] = 3 * _uniq[idx] - 2
        
        for i in range(1, _dof.shape[0], 3):
            idx = i // 3
            _dof[i] = 3 * _uniq[idx] - 1

        for i in range(2, _dof.shape[0], 3):
            idx = i // 3
            _dof[i] = 3 * _uniq[idx]

        edof = _dof[edof - 1]
        edof.resize((
            edof.shape[0], edof.shape[1]
        ))
        return edof.astype(int)

    def _compute_displacement(self, K, F, edof, ndof):
        activedofs = copy.deepcopy(edof)
        activedofs = np.sort(np.unique(self._flat_1d(activedofs)))
        activedofs -= 1

        end = activedofs.shape[0]
        K_sub = K[3:end, 3:end]
        L = self._ichol(K_sub)

        X = np.zeros((ndof, 6))
        for i in range(6):
            F_sub = F[3:end, i].todense()
            result, info = cg(A=K_sub, b=F_sub, tol=1e-10, maxiter=300, M=L * L.transpose())

            if info > 0:
                raise ValueError("Solver Failed")

            X[activedofs[3:end], i] = result

        return X

    def _flat_2d(self, v):
        v = copy.deepcopy(v)
        v = v.flatten('F')
        v.resize(v.shape[0], 1)
        return v

    def _flat_1d(self, v):
        v = copy.deepcopy(v)
        return v.flatten('F')

    def _write(self, filename, obj):
        np.savetxt(filename, obj, fmt="%f")

    def _ichol(self, A):
        solver = ilupp.IChol0Preconditioner(A)
        return solver.factors()[0]

    def _compute_hexahedron(self, a: float, b: float, c: float) -> Hexahedron:
        """Compute hexahedron from 3d voxel shape grid

        Args:
            a (int): Dimension A (x direction)
            b (int): Dimension B (y direction) 
            c (int): Dimension C (z direction)
        """
        # Constitutive matrix contributions - Mu
        C_mu = np.diag([2, 2, 2, 1, 1, 1])

        # Constitutive matrix contribution - Lambda
        C_lambda = np.zeros((6, 6))
        C_lambda[0:3, 0:3] = 1

        # Calculate the three gauss points in all 3 directions
        xx = np.array([-math.sqrt(3/5), 0, math.sqrt(3/5)])
        yy = np.array([-math.sqrt(3/5), 0, math.sqrt(3/5)])
        zz = np.array([-math.sqrt(3/5), 0, math.sqrt(3/5)])
        ww = np.array([5/9, 8/9, 5/9])

        # Initialize
        ke_lambda = np.zeros((24, 24))
        fe_lambda = np.zeros((24, 6))

        ke_mu = np.zeros((24, 24))
        fe_mu = np.zeros((24, 6))

        # Begin integration over the volume
        for i in range(len(xx)):
            for j in range(len(yy)):
                for k in range(len(zz)):
                    # Integration point
                    x = xx[i]
                    y = yy[j]
                    z = zz[k]

                    # Compute the stress-strain displacement matrix
                    qx = [
                        -((y-1)*(z-1))/8, 
                        ((y-1)*(z-1))/8, 
                        -((y+1)*(z-1))/8,
                        ((y+1)*(z-1))/8, 
                        ((y-1)*(z+1))/8,
                        -((y-1)*(z+1))/8,
                        ((y+1)*(z+1))/8, 
                        -((y+1)*(z+1))/8
                    ]

                    qy = [ 
                        -((x-1)*(z-1))/8,
                        ((x+1)*(z-1))/8,
                        -((x+1)*(z-1))/8,
                        ((x-1)*(z-1))/8,
                        ((x-1)*(z+1))/8, 
                        -((x+1)*(z+1))/8,
                        ((x+1)*(z+1))/8,
                        -((x-1)*(z+1))/8
                    ]
                    
                    qz = [
                        -((x-1)*(y-1))/8,
                        ((x+1)*(y-1))/8, 
                        -((x+1)*(y+1))/8,
                        ((x-1)*(y+1))/8, 
                        ((x-1)*(y-1))/8, 
                        -((x+1)*(y-1))/8,
                        ((x+1)*(y+1))/8, 
                        -((x-1)*(y+1))/8
                    ]
                    qq = np.array([qx, qy, qz])
                    dims = np.array([
                        [-a, a, a, -a, -a, a, a, -a],
                        [-b, -b, b, b, -b, -b, b, b],
                        [-c, -c, -c, -c, c, c, c, c],
                    ]).transpose()

                    # Compute the jacobian
                    J = np.matmul(qq, dims)

                    qxyz = np.linalg.lstsq(J, qq)[0]
                    B_e = np.zeros((8, 6, 3))

                    for ii in range(B_e.shape[0]):
                        B_e[ii] = np.array([
                            [qxyz[0, ii], 0, 0],
                            [0, qxyz[1, ii], 0],
                            [0, 0, qxyz[2, ii]],
                            [qxyz[1, ii], qxyz[0, ii], 0],
                            [0, qxyz[2, ii], qxyz[1, ii]],
                            [qxyz[2, ii], 0, qxyz[0, ii]],
                        ])

                    B = np.hstack([B_e[ii] for ii in range(B_e.shape[0])])

                    weight = np.linalg.det(J) * ww[i] * ww[j] * ww[k]

                    # Element matrices
                    ke_lambda += weight * np.matmul(np.matmul(B.transpose(), C_lambda), B)
                    ke_mu += weight * np.matmul(np.matmul(B.transpose(), C_mu), B)

                    # Element loads
                    fe_lambda += weight * np.matmul(B.transpose(), C_lambda)
                    fe_mu += weight * np.matmul(B.transpose(), C_mu)

        return Hexahedron(ke_lambda, ke_mu, fe_lambda, fe_mu)



In [506]:
# (For Later) https://stackoverflow.com/questions/19597473/binary-random-array-with-a-specific-proportion-of-ones

h = Homogenization(10, 10, 10, (10, 0), (10, 0), np.ones((10, 10, 10)))
h.homogenize()

[[ 3.00000000e+01  1.00000000e+01  1.00000000e+01  4.13558077e-14 -3.32879888e-14  2.69229083e-14]
 [ 1.00000000e+01  3.00000000e+01  1.00000000e+01  5.20140017e-14 -5.05151476e-14  2.35210034e-15]
 [ 1.00000000e+01  1.00000000e+01  3.00000000e+01  4.27876268e-14 -4.46916161e-14  2.00109138e-14]
 [ 4.03259471e-14  5.16242357e-14  4.29821513e-14  1.00000000e+01  6.66133815e-15 -5.55111512e-16]
 [-3.29293436e-14 -5.05361520e-14 -4.41849365e-14  6.15747872e-15  1.00000000e+01  2.40203670e-15]
 [ 2.57007541e-14  1.82064442e-15  1.96833091e-14 -1.94289029e-15  2.09748695e-15  1.00000000e+01]]
Homogenization complete!


<ipython-input-505-b3a05aabcdd7>:375: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  qxyz = np.linalg.lstsq(J, qq)[0]
<ipython-input-505-b3a05aabcdd7>:87: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  X0_e[indices, :] = np.linalg.lstsq(ke[indices, :][:, indices], fe[indices, :])[0]


In [378]:

x = 1
y = 2
z = 3
qx = [
    -((y-1)*(z-1))/8, 
    ((y-1)*(z-1))/8, 
    -((y+1)*(z-1))/8,
    ((y+1)*(z-1))/8, 
    ((y-1)*(z+1))/8,
    -((y-1)*(z+1))/8,
    ((y+1)*(z+1))/8, 
    -((y+1)*(z+1))/8
]

qy = [ 
    -((x-1)*(z-1))/8,
    ((x+1)*(z-1))/8,
    -((x+1)*(z-1))/8,
    ((x-1)*(z-1))/8,
    ((x-1)*(z+1))/8, 
    -((x+1)*(z+1))/8,
    ((x+1)*(z+1))/8,
    -((x-1)*(z+1))/8
]

qz = [
    -((x-1)*(y-1))/8,
    ((x+1)*(y-1))/8, 
    -((x+1)*(y+1))/8,
    ((x-1)*(y+1))/8, 
    ((x-1)*(y-1))/8, 
    -((x+1)*(y-1))/8,
    ((x+1)*(y+1))/8, 
    -((x-1)*(y+1))/8
]

qq = np.array([qx, qy, qz])
J = np.identity(3)
np.linalg.solve(J, qq)

qxyz = np.linalg.solve(J, qq) 
B_e = np.zeros((8, 6, 3))

for i in range(B_e.shape[0]):
    B_e[i] = np.array([
        [qxyz[0, i], 0, 0],
        [0, qxyz[1, i], 0],
        [0, 0, qxyz[2, i]],
        [qxyz[1, i], qxyz[0, i], 0],
        [0, qxyz[2, i], qxyz[1, i]],
        [qxyz[2, i], 0, qxyz[0, i]],
    ])

np.set_printoptions(edgeitems=10, linewidth=100000)
B = np.array([B_e[i] for i in range(B_e.shape[0])])
np.hstack([B_e[i] for i in range(B_e.shape[0])])

n_el_x = 100
n_el_y = 100
n_el_z = 100

print(n_el_x * n_el_y * n_el_z)
node_numbers = np.c_[np.arange(1, ((1 + n_el_x) * (1 + n_el_y) * (1 + n_el_z)) + 1)]
node_numbers = node_numbers.reshape(
    1 + n_el_x, 1 + n_el_y, 1 + n_el_z)

A = np.diag([1, 1, 1, 1])
B = np.array([[1, -1], [-1, 1]])
x = np.kron(A, B)

voxel = np.ones((10, 10, 10))
lambda_ = 10 * (voxel == 1)
lambda_.reshape((1, np.product(lambda_.shape)))

a = np.array([[1, 2]])
b = np.array([[3, 4]])
np.concatenate((a, b), axis=0)

1000000


array([[1, 2],
       [3, 4]])